In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

# Configuration

In [2]:
data_path = '../data/MIT-BIH/'
minute_input = 10
minute_output = 10
window_input= 40*minute_input
window_out= 40*minute_input
train_size = 0.8
test_size = 1 - train_size
data_set = {
  0: "test",
  1: "train"
}
model_name_reg_phase = "LSTM"
model_name_cls_phase = "LSTM"
# length_ecg là độ dài 2 khoảng RR được fixed lúc tiền xử lý dữ liệu 
# (độ dài một dòng trong file excel, trừ cột cuối là nhãn lớp bệnh tim)
length_ecg = 187 

'''
Độ dài của input/output càng dài thì số lượng file không đáp ứng đủ để tạo một mẫu hợp lệ 
cho mô hình càng nhiều. Để đám bảo tính thống nhất nên sẽ dùng độ dài dài nhất của phần 
input/output trong quá trình thực nghiệm để là chuẩn từ đó loại các file bị thiếu này đều
ở những phần thực nghiệm input/output khác.
'''
missing_file_train = ['201_V1.csv', '102_V2.csv', '124_V4.csv', '112_V1.csv', '203_V1.csv', '116_V1.csv', '108_V1.csv', '207_V1.csv', '111_V1.csv', '200_V1.csv', '207_MLII.csv', '210_V1.csv', '202_V1.csv', '113_V1.csv', '214_V1.csv', '121_V1.csv', '109_V1.csv', '105_V1.csv', '107_V1.csv', '115_V1.csv', '208_V1.csv']
missing_file_test = ['213_V1.csv', '231_V1.csv', '228_V1.csv', '222_V1.csv', '232_V1.csv']

In [3]:
class Dataset:
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # read data
        X = self.data[i]
        y = np.concatenate(self.label[i])
        return X, y

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size,size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size= size

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)

    def __len__(self):
        return self.size //self.batch_size

# Training Regression

## Get data

In [4]:
def get_data(istrainset):    
    missing_file = []
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        if (filename not in (missing_file_train)) and (filename not in (missing_file_test)) and (filename != ".DS_Store"):
            f = os.path.join(directory, filename)
            if os.path.isfile(f):
                df = pd.read_csv(f, header=None)
                data=df.drop(columns=187)
                data=data.values
                # Số lượng lặp qua dữ liệu
                num_samples = len(data) - window_input - window_out 

                if(num_samples>0):
                    total_sample = total_sample + num_samples
                    for i in range(num_samples):
                        X_window = data[i:i+window_input]
                        y_window = data[i+window_input+window_out:i+window_input+window_out+1]


                        X.append(X_window)
                        y.append(y_window)
                else:
                    missing_file.append(filename)
    print("------🍒------")
    print(f"Num of file in {data_set[istrainset]} set can not use due to its missing of length: {len(missing_file)}")
    print(f"Number of sample: {len(y)}/{len(X)}/{total_sample}")
    print(f"Missing files: {missing_file}")
    return X,y

In [5]:
X_train, y_train = get_data(1)
X_test, y_test = get_data(0)

------🍒------
Num of file in train set can not use due to its missing of length: 0
Number of sample: 69069/69069/69069
Missing files: []
------🍒------
Num of file in test set can not use due to its missing of length: 0
Number of sample: 19797/19797/19797
Missing files: []


In [6]:
train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

In [7]:
train_loader = Dataloader(train_dataset, 16,len(train_dataset))
test_loader = Dataloader(test_dataset,16,len(test_dataset))

In [8]:
print(train_loader[0][0].shape)
print(train_loader[0][1].shape)

(16, 400, 187)
(16, 187)


## Build Model

In [17]:
def build_model_reg(model_name,epochs):
    # Tạo một mô hình LSTM
    if model_name == "LSTM":
        model = Sequential()
        model.add(LSTM(64,activation='relu' ,input_shape=(window_input, 187)))  # Đặt input_shape phù hợp với kích thước của mảng X_train
        model.add(Dense(187))# Đặt lớp Dense phù hợp với kích thước của mảng y_train

        # Compile model
        learning_rate = 0.01
        adam = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)    
        model.compile(loss='mean_squared_error',optimizer=adam, metrics=['accuracy'])
    return model

## Train Model

In [18]:
model_name = model_name_reg_phase
epochs = 2
model = build_model_reg(model_name, epochs = epochs)
model.fit(train_loader, validation_data=test_loader, verbose=1, epochs=epochs)
model.save(f"trained/{model_name}_Phase1_PastECG_FutureECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")
print("Saved model to disk")

Epoch 1/2
4316/4316 [==============================] - 389s 90ms/step - loss: 0.0349 - accuracy: 0.1057 - val_loss: 0.0420 - val_accuracy: 0.0858
Epoch 2/2
4316/4316 [==============================] - 360s 83ms/step - loss: 0.0400 - accuracy: 0.1088 - val_loss: 0.0412 - val_accuracy: 0.0858
Saved model to disk


/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# model = tf.keras.models.load_model(f"trained/PastECG_FutureECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")
# # Dự đoán trên tập kiểm tra
# y_pred = model.predict(test_loader)
# df = pd.DataFrame({
#    'y_true': test_loader[0][1][0],
#    'y_pred': y_pred[0]
#    })
# lines = df.plot.line()

# Training Classification

## Get data

In [9]:
def get_data_class(istrainset):
    directory = f"{data_path}{data_set[istrainset]}/"
    if istrainset == 1:
        X, y = [], []
        for filename in os.listdir(directory):
            if (filename not in (missing_file_train)) and (filename not in (missing_file_test)) and (filename != ".DS_Store"):
                f = os.path.join(directory, filename)
                if os.path.isfile(f):
                    df = pd.read_csv(f, header=None)
                    data=df.values

                    X_window = data[:,:-1]
                    y_window = data[:,-1]
                    X.append(X_window)
                    y.append(y_window)
        print(f"Number of sample: {len(y)}/{len(X)}")
        return X,y
    else:
        y = []
        for filename in os.listdir(directory):
            if (filename not in (missing_file_train)) and (filename not in (missing_file_test)) and (filename != ".DS_Store"):
                f = os.path.join(directory, filename)
                if os.path.isfile(f):
                    df = pd.read_csv(f, header=None)
                    data=df.values

                    y_window = data[window_input+window_out:,-1]
                    y.append(y_window)
        print(f"Number of sample: {len(y)}")
        return y

In [10]:
# Mô hình phân lớp vẫn phải được huấn luyện trên tập train
X_class_train, y_class_train = get_data_class(1)
X_class_train= np.concatenate(X_class_train, axis=0)
y_class_train= np.concatenate(y_class_train, axis=0)
y_class_train=y_class_train.astype(int)

#Mô hình phân lớp dự đoán trên chính giá trị được pred từ mô hình hồi qui ở pha trước
model = tf.keras.models.load_model(f"trained/{model_name_reg_phase}_Phase1_PastECG_FutureECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")
# Dự đoán trên tập kiểm tra
y_pred = model.predict(test_loader)
X_class_test = y_pred
y_class_test = get_data_class(0)
y_class_test=np.concatenate(y_class_test)
y_class_test=y_class_test.astype(int)

Number of sample: 55/55
1237/1237 [==============================] - 47s 38ms/step
Number of sample: 15


## Build Model

In [12]:
def build_model_cls(model_name,epochs):
    # Tạo một mô hình LSTM
    if model_name == "LSTM":
        input_shape = (1, length_ecg)
        model= Sequential()
        model.add(LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True))
        model.add(LSTM(64, activation='relu', return_sequences=True))
        model.add(LSTM(64, activation='relu', return_sequences=True))
        model.add(LSTM(64, activation='relu'))
        model.add(Dense(5, activation='softmax'))

        # Compile model
        learning_rate = 0.01
        adam = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)        
        model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [90]:
model_name = model_name_reg_phase
epochs = 10
model = build_model_cls(model_name_cls_phase,epochs)
model.fit(X_class_train.reshape(X_class_train.shape[0], 1, X_class_train.shape[1]), y_class_train, epochs=epochs, batch_size=32)
model.save(f"trained/{model_name}_Phase2_PastECG_FutureECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")
print("Saved model to disk")

Epoch 1/10
3534/3534 [==============================] - 8s 2ms/step - loss: 0.1749 - accuracy: 0.9567
Epoch 2/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1318 - accuracy: 0.9670
Epoch 3/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1258 - accuracy: 0.9695
Epoch 4/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1129 - accuracy: 0.9720
Epoch 5/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1184 - accuracy: 0.9707
Epoch 6/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1210 - accuracy: 0.9721
Epoch 7/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1405 - accuracy: 0.9657
Epoch 8/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1392 - accuracy: 0.9671
Epoch 9/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.1294 - accuracy: 0.9692
Epoch 10/10
3534/3534 [==============================] - 6s 2ms/step - loss: 0.125

# Script

In [ ]:
input_shape = (window_input, length_ecg)
model= Sequential()
model.add(LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.summary()
for layers in (model.layers)[:4]:
    print(layers)
    layers.trainable = False
model.summary()
model.layers